# Deep Hedging AI
### Vanilla Deep Hedging engine reference implementation with dynamic training update.
### This is the main example notebook

Hans Buehler, June 30 2022


In [1]:
""" Slighly annoying: by default the SageMaker Python import directory does not include our git directory """
import os
p = os.getcwd()
dhn = "/deephedging/"
i = p.find(dhn)
if i!=-1:
    p = p[:i]
    import sys
    sys.path.append(p)
    print("SageMaker: added python path %s" % p)

SageMaker: added python path /home/ec2-user/SageMaker


In [2]:
print("Deep Hedging AI says hello  ... ", end='')
from cdxbasics.config import Config
from deephedging.trainer import train
from deephedging.gym import VanillaDeepHedgingGym
from deephedging.world_bs import SimpleWorld_BS

from IPython.display import display, Markdown

# see print of the config below for numerous options
config = Config()
# world
config.world.samples = 10000
config.world.steps = 20
# gym
config.gym.objective.utility = "cvar"
config.gym.objective.lmbda = 1.
config.gym.agent.network.depth = 3
config.gym.agent.network.activation = "softplus"
# trainer
config.trainer.train.batch_size = None
config.trainer.train.epochs = 400
config.trainer.visual.epoch_refresh = 1
config.trainer.visual.time_refresh = 10
config.trainer.visual.confidence_pcnt_lo = 0.25
config.trainer.visual.confidence_pcnt_hi = 0.75

display(Markdown("## Deep Hedging in a Black \& Scholes World"))

# create world
world  = SimpleWorld_BS( config.world )
val_world  = world.clone(samples=1000)

# create training environment
gym = VanillaDeepHedgingGym( config.gym )

# create training environment
train( gym=gym, world=world, val_world=val_world, config=config.trainer )
r = gym(world.tf_data)
print("Keys of the dictionary returned by the gym: ", r.keys())

print("=========================================")
print("Config usage report")
print("=========================================")
print( config.usage_report() )
config.done()

Deep Hedging AI says hello  ... Tensorflow version 2.7.1 running on 1 CPUs and 0 GPUs


2023-01-10 11:56:46.044838: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-10 11:56:46.044879: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-71-57.eu-west-2.compute.internal): /proc/driver/nvidia/version does not exist


## Deep Hedging in a Black \& Scholes World

PrettyOrderedDict([('per_step', PrettyOrderedDict([('cost', array([[0.0002    , 0.00020218, 0.00019521, ..., 0.00020334, 0.00020672,
        0.00020933],
       [0.0002    , 0.00019342, 0.00019433, ..., 0.00018494, 0.0001843 ,
        0.0001895 ],
       [0.0002    , 0.00020716, 0.00021661, ..., 0.00018881, 0.00018998,
        0.00019583],
       ...,
       [0.0002    , 0.00020391, 0.00021052, ..., 0.00028224, 0.00027952,
        0.00027691],
       [0.0002    , 0.00019887, 0.00019277, ..., 0.0001914 , 0.00018236,
        0.00017866],
       [0.0002    , 0.0001957 , 0.00019763, ..., 0.00019545, 0.00020158,
        0.0002092 ]])), ('price', array([[1.        , 1.01088573, 0.9760424 , ..., 1.0335852 , 1.04664751,
        1.01536461],
       [1.        , 0.9671102 , 0.97166023, ..., 0.92152483, 0.94752227,
        0.9663125 ],
       [1.        , 1.03582261, 1.08305009, ..., 0.94989317, 0.97916934,
        0.97335389],
       ...,
       [1.        , 1.02144335, 1.00747984, ..., 1.009516

2023-01-10 11:56:46.650658: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


IndexError: list index out of range

### Comparison to Black & Scholes
#### See also notebooks/blackscholes

Note that we use medium risk aversion, hence the hedge is not expected to match the risk-neutral hedge. See the sub directory <tt>blackscholes</tt> for experiments approximating the black & scholes solution

In [ ]:
import deephedging.plot_bs_hedge as plot_bs_hedge
import imp
imp.reload(plot_bs_hedge)

plot_bs_hedge.plot_blackscholes( world, gym, config )